# The Laplace Beltrami Operator, Spectral Mesh Analysis and Geodesics
In this exercise, you will first build the Laplace-Beltrami operator and subsequently apply it to several tasks ranging from smoothing of meshes to decomposing a mesh in terms of the eigenvectors of the operator (the spectral analysis) to computing shortest distances on a mesh.

In [ ]:
from pygel3d import hmesh, jupyter_display as jd, gl_display as gl
jd.set_export_mode(True)
from numpy import zeros, eye, array, linalg, transpose, dot, cross, sin, exp
from scipy.linalg import eigh, solve
from itertools import permutations, combinations
from math import tan, acos, cos, sqrt
from random import random
from scipy.spatial import KDTree

## Part 1: Create the Laplace Beltrami Operator
Below your task is to create the LBO as a matrix given an input mesh. Use the cotan formula from the slides and course notes. In this part, you should normalize each row with the 1-ring area of the corresponding vertex.

In [ ]:
def normalize(v):
    return v / linalg.norm(v)

# The function below is a helper function. The idea is that you give it a mesh and
# three vertices from a triangle. i and j define the edge, and k is the vertex at which 
# you compute cotan of the angle. 
def cotan_angle(m,i,j,k):
    
    # Your code here
    # ---------------------------

# This function is passed a mesh and returns the cotan LBO 
def create_laplacian(m, symmetric=False):
    # symmetric is a flag that you can choose in order to make the matrix L symmetric or not
    no_verts = m.no_allocated_vertices()
    L = zeros((no_verts,no_verts)) # The cot Laplacian that we end up outputting 
    A = zeros((no_verts)) # Use this vector to store 1-ring areas
    
    # Your code here
    # ---------------------------
    
    
    
    return L

# Load the mesh and make safe copy of vertex positions
m = hmesh.load("data/bunnygtest.obj")
pos = m.positions()
pos_orig = array(pos)
N = m.no_allocated_vertices()

## Part 2: Smooth the Mesh using the LBO

In [ ]:
# Adding noise
pos_noisy = pos_orig[:] + 0.0025* array([[random(),random(),random()] for _ in range(0,N)])
pos[:] = pos_noisy

# Show noisy mesh
jd.display(m,smooth=False)

In [ ]:
# Explicit smoothing.
#
# Explicit smoothing is simply a matrix multiplication. Try several step sizes and 
# note how the effect changes. Also, try to perform several iterations of smoothing.
#

# Your code here
# ---------------------------

jd.display(m,smooth=False)

# Resetting to noisy positions
pos[:] = pos_noisy

In [ ]:
# Implicit smoothing
#
# Implicit smoothing requires solving a linear system. Try several step sizes
# and note the differences compared to explicit smoothing.

# Your code here
# ---------------------------

jd.display(m,smooth=False)

# Resetting to noisy positions
pos[:] = pos_noisy



## Part 3: Compute the Eigenvectors and Eigenvalues of the LBO
You need to slightly change the code that computes the Laplace Beltrami Operator to obtain a symmetric matrix suitable for eigendecomposition. Use the formula in 1.3.2. This code also visualizes some of the eigenvectors. 

In [ ]:
#Reset positions to non-noisy values
pos[:] = pos_orig

# Compute the symmetric Laplacian
L = #

# Next, find the eigenvectors and eigenvalues of the symmetric Laplacian
# In fact, we need to use -L because that way the eigenvectors are positive.

# Your code here
# ---------------------------

(W,V) = #

# Now display some of the eigenvectors. The code below uses the OpenGL viewer
# uncomment to use.

#viewer = gl.Viewer()
#for i in range(20):
#    print(W[i])
#    viewer.display(m,mode='s',smooth=True,data=V[:,i])
#del viewer
jd.display(m,data=V[:,2])

## Part 4: Project the Positions onto the LBO Eigenvectors
This is the actual spectral analysis where we do something similar to the Fourier transform for images by projecting the vertex positions onto eigenvectors of the LBO.

In [ ]:
# Function that analyses a signal by projecting onto the eigenvectors
def analyze_signal(V, sig):
    
    # Your code here
    # ---------------------------
    
    return #
    
# Reconstruct a signal by taking linear combinations of eigenvectors
def recon_signal(V, sig_hat):
    
    # Your code here
    # ---------------------------
    
    return #


# Projecting positions onto Laplace-Beltrami operator eigenvectors
pos_hat = analyze_signal(V,pos)

# Showing iterative reconstruction
# The viewer below will show an animation of the reconstruction, and you
# may want to uncomment.
#viewer = gl.Viewer()
#viewer.display(m,'w',smooth=True,reset_view=True)
for i in range(50):
    mask = array([[1,1,1]]*i+[[0,0,0]]*(N-i))
    pos[:] = recon_signal(V,pos_hat*mask)
#    viewer.display(m,'w',smooth=True, once=True)

# Display result after reconstruction with 50 eigenvectors
jd.display(m)
# Resetting positions
pos[:] = pos_orig
#del viewer

## Part 5: Boost the high frequencies.
In this part, we boost the high frequencies, creating a caricature of the original mesh. Experiment with both the magnitude and what frequencies that you boost.

In [ ]:
# Boosting high frequencies
#viewer = gl.Viewer()

# Compute here boosting of high frequencies
pos_hat = analyze_signal(V,pos_orig)

# Your code here
# ---------------------------


pos[:] = recon_signal(V,pos_hat)

#viewer.display(m,'w',smooth=True)
jd.display(m)

# Resetting positions
pos[:] = pos_orig
#del viewer

In [ ]:
# Boosting high frequencies
#viewer = gl.Viewer()
pos_hat = analyze_signal(V,pos_orig)

# Your code here
# ---------------------------


pos[:] = recon_signal(V,pos_hat)

#viewer.display(m,'w',smooth=True)
jd.display(m)

# Resetting positions
pos[:] = pos_orig
#del viewer

## Part 6: Geodesics with Laplace Beltrami Operator
In this part of the exercise, we use the Laplace Beltrami Operator to compute the geodesic distance to a source vertex for each vertex of the mesh. The geodesic distance from a point on the mesh to the source vertex is the distance along the shortest geodesic that connects the point to the source vertex. The method that you implement in this exercise is based on the paper: **"The Heat Method for Distance Computation"** by Crane, Weischedel and Wardetzky (2013), and the basic idea is to first solve the heat equation and then find a distance field whose gradients are (approximately) aligned with the gradients of the solution to the heat equation.

Specifically, this part of the exercise is a 3-step process, where we: 

1) Integrate the heat flow $\dot{u} = \Delta u$ where the initial condition is a $\delta$ function placed at the source vertex.

2) Evaluate the vector field $X = - \nabla u_t / |\nabla u_t|$

3) Solve the Poission euqation $\Delta \phi = \nabla \cdot X$



### Part 6.1
This part corresponds to step 1). You should set the time step $t$ and then integrate $u$, which corresponds to the step 1) in the introduction. **Note:** $u \in \mathbb{R}^{|V|}$. 

In [ ]:
m = hmesh.load("bunnygtest.obj")

A crucial part of the heat distance method is setting the time step $t$. A simple estimate that works well is: $t =k \cdot h^2$, where $h$ is the average edge length and $k$ is a suitable constant.

In [ ]:
# Setting the time step t. Experiment with this number 

# Your code here
# ---------------------------
t = # 

In [ ]:
# Setting a source vertex from where heat should emerge. 
source_vertex = 836

# Or pick the source vertex interactively by uncommenting the code below.
# Hit "Crtl" and click on the point that you want. Then afterwards hit Escape.

#viewer = gl.Viewer()
#viewer.display(m)
#tree = KDTree(m.positions())
#_, index = tree.query(viewer.annotation_points(), k=1)
#source_vertex = index[0]
#del viewer

Now, you need to solve the heat equation where the initial value of $u$ is a delta function placed at the source vertex, $x$. Fortunately, we know that the solution to the heat equation where $u(0) =\delta_{x}$ is simply the heat kernel placed at $x$. The heat kernel we can express in terms of LBO eigenvectors and eigenvalues,

$u \approx \sum_{i=1}^N \exp{(-t \cdot \lambda_i)} \cdot \mathbf{e}_{i,x} \cdot \mathbf{e}_i$,

where $\lambda_i$ is the i'th eigenvalue and $\mathbf{e}_{i,x}$ is the source vertex'th entry of eigenvector $\mathbf{e}_{i}$. $N$ is a suitable number of eigenvalues/eigenvectors. Note that the bigger $t$ is, the less high-numbered eigenvectors count (why?).

In [ ]:
def u_integrated(W, V, N, t, source_vertex):
    # Your code here
    # ---------------------------
    return u_vector

In [ ]:
N = len(W)
u = u_integrated(W, V, N, t, source_vertex)

### Part 6.2
In this part you need to compute the vector field $X = -\nabla u_t / |\nabla u_t|$ and from this compute the divergence $\nabla \cdot X$. This corresponds to step 2) in the introduction.

**Note:** $X$ is a $3\times 1$ vector.

In [ ]:
# Compute the 3x1 vector X at a triangle face. 
# The input to the function is the mesh m, the solution u from Part 2 and the index f of a given triangle face
def computeVector(m,u,f):
    # Your code Here 
    #-------------------------------
    
    return X

In [ ]:
# This function computes the divergence (nabla X) at each vertex.
# The input to the function is the mesh m and the solution u from Part 2.
# The function should return the |V|x1 vector b
def computeDivergence(m,u):
    pos = m.positions()
    b = zeros(m.no_allocated_vertices())
    for v in m.vertices():
        for h in m.circulate_vertex(v,mode='h'):
            e1 = pos[m.incident_vertex(h)] - pos[v]
            e2 = pos[m.incident_vertex(m.next_halfedge(h))] - pos[v]
            vec1 = pos[m.incident_vertex(m.next_halfedge(h))] - pos[m.incident_vertex(h)]
            theta2 = acos(dot(-e1,vec1)/(linalg.norm(-e1)*linalg.norm(vec1)))
            theta1 = acos(dot(-e2,-vec1)/(linalg.norm(-e2)*linalg.norm(-vec1)))
            cot_theta2 = 1.0/tan(theta2)
            cot_theta1 = 1.0/tan(theta1)
            Xj = computeVector(m, u, m.incident_face(h))
            b[v] += cot_theta1 * dot(e1, Xj) + cot_theta2 * dot(e2, Xj)
        b[v] *= 0.5
    return b

### Part 6.3
In this part of the exercise, you are supposed to create the matrix $L_c$ and then solve the linear system of equations $L_c \phi = b$ in order to find the geodesic distances to the other vertices. This corresponds to step 3) in the introduction. **Note:** $b \in \mathbb{R}^{|V|}$ and the i'th entry in $b$ is the divergence $\nabla \cdot X$ at the i'th vertex in the mesh. 

In [ ]:
# This function is passed a mesh and returns the cotan operator matrix L_c
def create_cotan_laplacian(m):
    no_verts = m.no_allocated_vertices()
    # The cot Laplacian that we end up outputting. It should be the same as before but without the weight 
    # normalization
    Lc = zeros((no_verts,no_verts)) 
    

    # Your code here
    # ---------------------------
    return Lc

In [ ]:
# Create the Laplace Cotangent Matrix
Lc = create_cotan_laplacian(m)

In [ ]:
b = computeDivergence(m,u)

In [ ]:
phi = linalg.solve(Lc,b)

**Note:** Now we have obtained $\phi$, but the solution to $\Delta \phi = \nabla \cdot X$ is only determined up to an additive constant (Why?). Therefore, we simply need to shift $\phi$, such that the minimum (located at the source vertex) is at $0.0$.

In [ ]:
# Your code here
# ---------------------------
print("Distance at source vertex is:", phi[source_vertex])

In [ ]:
# Code to show the heat kernel. 
# You might need to play around with the constant times u
#jd.display(m, data=np.sin(u*2))

In [ ]:
# Plot the result - You should see rings of equal size that display euqal distance to the source vertex
# We multiply with the constant only to show the periodic pattern.
#jd.display(m, data=phi)
jd.display(m, data=sin(phi*300))

### Part 7: Questions
- Explain the difference in principle and effect between implicit and explicit smoothing.
> Answer:
- Explain how smoothing with the LBO operator is different from simply assigning the average positions of neighbors to a vertex. (an operator that is sometimes called the umbrella operator),
> Answer:
- Explain what the first two eigenvectors of the LBO represent (eigenvector 0 and 1).
> Answer:
- Explain what boosting of high frequencies does to the geometry.
> Answer: 
- Why is the geodesic distance different from the Euclidean distance?
> Answer:
- Given the distance map computed in 6.3, how would we find a geodesic curve that connects an arbitrary point back to the source vertex?
> Answer
- Why does The Heat Method for computing distances above only approximate the true geodesic distance?
> Answer